# DATACHALLENGE

Pour ce datachallenge, nous aurons besoin des modules Python ci-dessous, il vous faut donc évidemment exécuter cette première cellule.

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn
from tensorflow import keras

Les différentes méthodes dont nous aurons besoins

### Pour importer les fichiers csv
pd.read_csv

### Pour créer nos echantillons de train et de test
from sklearn import model_selection.train_test_split

model_selection.train_test_split

### Trouver la meilleur valeur d'un paramètre
form sklearn import model_selection

model_selection.GridSearchCV

### Créer une répartition d'échantillon de test aléatoire
from sklearn import model_selection

model_selection.KFold

### Pour calculer une regression linéaire
from sklearn import linear_model

linear_model.LogisticRegression

### Pour faire une PCA
from sklearn import decomposition

decomposition.PCA

### Classificatuer gaussien
from sklearn.naive_bayes import GaussianNB

### Classificateur k-voisins
from sklearn import neighbors

neighbors.KNeighborsClassifier

### Regression logistique
from sklearn import linear_model

linear_model.LogisticRegression

### Regression logistique lasso
from sklearn import linear_model

linear_model.LogisticRegressionCV

### Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

### Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor

### Bagged Tree
from sklearn.ensemble import BaggingRegressor

### Random forest regressor
from sklearn.ensemble import RandomForestRegressor

### Reseaux de neurones
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Activation

from tensorflow.keras.utils import plot_model

from tensorflow.keras.optimizers import SGD

# Importation des données

Nous allons importer toutes les données qui vont nous être utiles, même celles qui ne seront pas utiles avant la fin


In [2]:
XTrain_Path = "Xtrain_hgcGIrA.csv"
YTrain_Path = "Ytrain_yL5OjS4.csv"
XTest_Path = "Xtest.csv"

In [37]:
XTrain  =    pd.read_csv(XTrain_Path)
YTrain  =    pd.read_csv(YTrain_Path)["p0q0"]
XTest  =    pd.read_csv(XTest_Path)

In [6]:
XTrain.head()

,date,train,way,station,hour,composition,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3
0,2019-01-07,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.201,0.138,0.091
1,2019-01-08,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.204,0.152,0.106
2,2019-01-10,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.153,0.111
3,2019-01-11,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.213,0.152,0.108
4,2019-01-14,1,0,AD,06:00:00,2,NaN,NaN,NaN,0.210,0.147,0.096


# Exemple de Découpage train / test

En apprentissage statistique, classiquement un prédicteur est ajusté sur une partie seulement des données et l'erreur de ce dernier est ensuite évaluée sur une autre partie des données disponibles. Ceci permet de ne pas utiliser les mêmes données pour ajuster et évaluer la qualité d'un prédicteur. Cette problématique est l'objet du prochain chapitre.

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), on sélectionne aléatoirement 60% des observations pour l'échantillon d'apprentissage et on garde le reste pour l'échantillon de test. 

In [12]:
from sklearn import model_selection

XTrain_train,XTrain_test,YTrain_train, YTrain_test = model_selection.train_test_split(XTrain.to_numpy(), YTrain.to_numpy(), test_size=0.4, train_size=0.6, random_state=0, shuffle=True, stratify=None)

# Exemple de transformation des predictions en CSV

Il faut que l'on obtienne un fichier csv pour tester nos résultats, voici comment on procède

In [91]:
prediction = YTrain.to_numpy()
print(prediction)

df = pd.DataFrame(prediction, columns=['p0q0'])
df.to_csv("predictionExemple.csv")

[0.216 0.216 0.227 ... 0.139 0.117 0.416]


# Decision tree regressor avec pq

In [95]:
#Selections des colonnes
XTrain_ = XTrain[["p0q1","p0q2", "p0q3", "p1q0","p2q0","p3q0"]]

#Nan -> 0
XTrain_ = XTrain_.fillna(0)

#Découpage train/test
from sklearn import model_selection
XTrain_train,XTrain_test,YTrain_train, YTrain_test = model_selection.train_test_split(XTrain_.to_numpy(), YTrain.to_numpy(), test_size=0.4, train_size=0.6, random_state=0, shuffle=True, stratify=None)


#Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor

DTR=DecisionTreeRegressor().fit(XTrain_.to_numpy(), YTrain.to_numpy())
prediction = DTR.predict(XTest_.to_numpy())


## Rendu première méthode

In [52]:
#selection des colonnes
XTrain_ = XTrain[["p0q1","p0q2", "p0q3", "p1q0","p2q0","p3q0"]]
XTrain_ = XTrain_.fillna(0)
XTest_ = XTest[["p0q1","p0q2", "p0q3", "p1q0","p2q0","p3q0"]]
XTest_ = XTrain_.fillna(0)

#Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor

DTR=DecisionTreeRegressor().fit(XTrain_.to_numpy(), YTrain.to_numpy())
prediction = DTR.predict(XTest_.to_numpy())


df = pd.DataFrame(prediction, columns=['p0q0'], index = np.arange(1, prediction.size+1))
df.to_csv("prediction1.csv")


# Decision tree regressor avec pq + sens, train, heure, jour, composition

In [48]:
#transformation en float des colonnes date et hour
from datetime import datetime
XTrain_hd = XTrain[["hour","date", "way", "train", "composition"]]
XTrain_hd = XTrain_hd.fillna(method='ffill')
hour_f = np.zeros((XTrain_hd.shape[0]))
day_f = np.zeros((XTrain_hd.shape[0]))

for i in range(XTrain_hd.shape[0]):
    hour_f[i] = (datetime.strptime(XTrain_hd["hour"][i],'%H:%M:%S')-datetime.strptime('00:00:00','%H:%M:%S')).seconds
    day_f[i] = datetime.strptime(XTrain_hd["date"][i],"%Y-%m-%d").weekday()
    
#récupération des colonnes
XTrain_ = XTrain[["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"]]
XTrain_ = XTrain_.fillna(0)

XTrain_["hour"]=hour_f
XTrain_["day"]=day_f
XTrain_["way"]=XTrain_hd["way"]
XTrain_["train"]=XTrain_hd["train"]
XTrain_["composition"]=XTrain_hd["composition"]






#même opération pour XTest

XTest_hd = XTest[["hour","date", "way", "train", "composition"]]
XTest_hd = XTest_hd.fillna(method='ffill')
hour_f = np.zeros((XTest_hd.shape[0]))
day_f = np.zeros((XTest_hd.shape[0]))

for i in range(XTest_hd.shape[0]):
    hour_f[i] = (datetime.strptime(XTest_hd["hour"][i],'%H:%M:%S')-datetime.strptime('00:00:00','%H:%M:%S')).seconds
    day_f[i] = datetime.strptime(XTest_hd["date"][i],"%Y-%m-%d").weekday()
    
#récupération des colonnes
XTest_ = XTest[["p1q0","p2q0","p3q0","p0q1","p0q2","p0q3"]]
XTest_ = XTest_.fillna(0)

XTest_["hour"]=hour_f
XTest_["day"]=day_f
XTest_["way"]=XTest_hd["way"]
XTest_["train"]=XTest_hd["train"]
XTest_["composition"]=XTest_hd["composition"]



In [51]:
#Calcul de modèle de décision tree regressor
#FAIRE DES TEST EN CHANGEANT LE CALCUL DE SCORE
#Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor

DTR=DecisionTreeRegressor().fit(XTrain_.to_numpy(), YTrain.to_numpy())
prediction = DTR.predict(XTest_.to_numpy())


df = pd.DataFrame(prediction, columns=['p0q0'], index = np.arange(1, prediction.size+1))
df.to_csv("prediction2.csv")



# Bagged Tree Regressor

In [53]:
from sklearn.ensemble import BaggingRegressor

#utiliser gridSearchCV pour trouver le nombre d'estimateur optimal
